[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//incubodac/GradCamTP/blob/main/GradCamTP.ipynb)

# Índice

1. **Introducción**
   a. Explicaciones visuales en modelos de redes profundas  
2. **Antecedentes**  
   a. Oclussion Maps  
   b. Guided Backpropagation  
   c. CAM  
3. **GradCAM**  
   a. Explicación  
   b. Ejemplos clasificación: ResNet50, VGG16, etc.  
   c. Ejemplos otras tareas: Automatic Captioning  
   d. Ventajas y desventajas  
4. **Mejoras: GradCAM++ y otros métodos**
5. **Referencias:**  

___

# 1. Introducción

La explicabilidad en redes neuronales convolucionales (CNN) ha surgido como un aspecto clave en el campo del reconocimiento de patrones, donde no solo interesa alcanzar un alto desempeño, sino también comprender qué información utiliza la red para tomar decisiones. Un primer avance clave fueron los **Class Activation Mapping (CAM, 2016)**, que permitieron visualizar las regiones más relevantes de una imagen para la clasificación, aunque con la limitación de aplicarse únicamente a arquitecturas convolucionales específicas que incluyan un *global average pooling* previo a la capa de clasificación densa. Sin embargo, no fue hasta que apareció **Grad-CAM (2017)** que amplió esta idea utilizando gradientes retropropagados, logrando un método versátil aplicable a una amplia variedad de arquitecturas y consolidándose como la técnica de referencia en explicabilidad. A partir de allí se desarrollaron extensiones como **Grad-CAM++ (2018)**, que mejora la localización en escenarios complejos, o **Score-CAM (2019)**, que evita depender directamente de los gradientes. El salto conceptual decisivo que abrió la puerta al estudio moderno de la explicabilidad fue el de Grad-CAM y en este trabajo pretendemos dar un panorama al respecto, analizando sus fundamentos, aplicaciones y comparativas con otros métodos.

# 2. Antecedentes

Uno de los grandes desafíos en el uso de redes neuronales profundas es su naturaleza de "caja negra", que dificulta entender cómo y por qué toman ciertas decisiones. Aunque hoy en día se entiende mucho más sobre su funcionamiento sobretodo en el caso de las convolucionales, la interpretabilidad sigue siendo un área activa de investigación. En respuesta a esta problemática, se han desarrollado diversas técnicas para generar explicaciones visuales que ayuden a interpretar el comportamiento de estos modelos. A continuación, se describen brevemente algunas de las técnicas más relevantes que preceden (e influencian) a Grad-CAM:

### Occlusion Maps

**Occlusion Map** (Zeiler, et al. 2014) es uno de los métodos más intuitivos y directos para generar explicaciones visuales en redes neuronales. La idea central consiste en evaluar la sensibilidad de la predicción de la red cuando partes de la entrada son ocultadas o reemplazadas por un valor neutro (por ejemplo, un parche negro o ruido). En la práctica, se recorre la imagen aplicando una “ventana de oclusión” que cubre sistemáticamente diferentes regiones, y se mide cómo varía la probabilidad asociada a la clase objetivo. Las zonas donde la predicción se ve más afectada se interpretan como las más relevantes para la decisión del modelo.  

Este enfoque tiene varias ventajas: es sencillo de implementar, no depende de modificaciones en la arquitectura del modelo ni de los gradientes, y produce mapas de importancia fáciles de interpretar. No obstante, también presenta desventajas: es computacionalmente costoso (ya que requiere múltiples inferencias por imagen), la resolución depende del tamaño del parche usado, y no captura bien interacciones complejas entre diferentes regiones de la entrada. A pesar de estas limitaciones, Occlusion Maps es un método de referencia en la literatura y se utiliza frecuentemente como baseline en estudios de explicabilidad (ver por ejemplo Samek et al. 2019)



### Guided Backpropagation

**Guided Backpropagation** (Springenberg et al., 2014, Montavon et al., 2018) es un método de visualización que busca identificar qué regiones de una imagen influyen positivamente en la activación de una clase objetivo dentro de una red neuronal profunda. A diferencia del backpropagation estándar, este enfoque modifica el flujo de gradientes en las funciones de activación ReLU, permitiendo únicamente el paso de aquellos gradientes positivos asociados a activaciones positivas. El resultado es un mapa de sensibilidad de alta resolución que resalta bordes y detalles finos de la entrada, mostrando qué píxeles contribuyen a la predicción. Sin embargo, no es estrictamente clase-específico, ya que puede resaltar patrones que activan la red independientemente de la salida seleccionada. Por este motivo, suele utilizarse en combinación con Grad-CAM, logrando visualizaciones más interpretables y centradas en la clase de interés.


### CAM

**CAM** fue introducido por Zhou et al. (2016) y se basa en la utilización de una capa de *global average pooling (GAP)* antes de la capa final de clasificación. La idea es que los pesos de la capa de salida pueden emplearse para ponderar los mapas de activación de la última capa convolucional, generando así un mapa de calor que indica las regiones de la imagen más relevantes para una clase determinada.

Se contruyen los mapas de activacion para cada clase de interes como:

$$
M_c(x,y) = \sum_k w_k^c f_k(x,y)
$$

donde:

* $M_c(x,y)$ es el mapa de activación para la clase $c$,
* $f_k(x,y)$ es el mapa de activación de la característica $k$,
* $w_k^c$ es el peso asociado a la clase $c$ en la capa de salida.


Además de requerir global average pooling, CAM solo puede aplicarse a capas cuya última capa sea una red densa. Un uso interesante es la posibilidad de hacer localizacion de objetos mediante una red que no haya sido entrenada con bounding boxes.

# 3. Grad-CAM


### Metodología

<!-- **(CONVERTIR PRINCIPALMENTE EN UN PARRAFO - AGREGAR UNA INTUICIÓN)**
Grad-CAM es una técnica que permite visualizar las regiones de una imagen que son relevantes para la decisión de un modelo de red neuronal. A través del uso de gradientes, se puede obtener un mapa de calor que indica qué partes de la imagen contribuyen más a la activación de una clase específica. Esto se logra calculando los gradientes de la clase de interés respecto a los mapas de activación de una capa convolucional, lo que permite ponderar las características de acuerdo con su importancia para la predicción.

**Aporte:**
Grad-CAM, propuesto por Selvaraju et al. (2017), generaliza la idea de CAM para que pueda aplicarse a una gran variedad de arquitecturas. Utiliza los gradientes de la clase de interés respecto a los mapas de activación de una capa convolucional, ponderando las características de acuerdo con su importancia para la predicción. -->

Grad-CAM es una técnica que permite visualizar qué regiones de una imagen son más relevantes para la decisión de un modelo de red neuronal. La idea central es aprovechar los gradientes de la clase de interés respecto a los mapas de activación de una capa convolucional para generar un mapa de calor que indique qué partes de la entrada influyen más en la predicción. Intuitivamente, esto equivale a preguntarse: si modifico esta región de la imagen, cuánto cambia la confianza del modelo en la clase que me interesa? De esta manera, se obtienen explicaciones visuales que ayudan a interpretar cómo la red toma sus decisiones.

Propuesto por Selvaraju et al. (2017), Grad-CAM generaliza la técnica CAM para que pueda aplicarse a una mayor variedad de arquitecturas, sin depender de capas global average pooling. El método consiste en calcular los gradientes de la clase de interés respecto a los mapas de activación, promediarlos para obtener un peso por canal, y finalmente combinarlos para formar un mapa de activación ponderado.

**Fórmula principal:**

Dada una instancia de entrada y una clase de interés $c$, para calcular el mapa de explicación Grad-CAM se siguen los siguientes pasos:

1. Elegimos una capa convolucional de interés (en general la más cercana a la salida) y calculamos los gradientes asociados a la clase $y^c$ respecto a los mapas de activación $A^k$ de dicha capa:

$$
\alpha_k^c = \frac{1}{Z} \sum_i \sum_j \frac{\partial y^c}{\partial A_{ij}^k}
$$

2. Generar el mapa de activación como:

$$
L_{\text{Grad-CAM}}^c = ReLU\left(\sum_k \alpha_k^c A^k \right)
$$

donde $ReLU$ asegura que solo se conserven las contribuciones positivas.

La intuición detrás de Grad-CAM es que los gradientes proporcionan una medida de sensibilidad, indicando qué tan importante es cada característica para la predicción de la clase objetivo. Al combinar esta información con los mapas de activación, se obtiene una visualización que resalta las regiones de la imagen que más influyen en la decisión del modelo.

<!-- **Ventajas:**

* Se puede aplicar a múltiples arquitecturas (sin necesidad de GAP).
* Permite obtener explicaciones visuales interpretables en tareas de clasificación, detección y segmentación.

**Limitaciones:**

* La resolución espacial del mapa depende de la capa elegida (generalmente baja).
* Tiende a resaltar regiones amplias sin un detalle fino de contornos. -->

**Ventajas y limitaciones**

Grad-CAM es una técnica muy versátil, ya que puede aplicarse a una amplia gama de arquitecturas de redes neuronales sin requerir modificaciones específicas como el uso de capas global average pooling. Esto le da la capacidad de generar explicaciones visuales interpretables en distintos tipos de tareas, incluyendo clasificación de imágenes, detección de objetos, automatic captioning o segmentación semántica, lo que lo convierte en una herramienta de gran valor para comprender y depurar modelos complejos. Aun así, presenta ciertas limitaciones: la resolución espacial de los mapas de calor depende fuertemente de la capa convolucional elegida, lo que suele implicar representaciones relativamente gruesas con poco detalle local. Además, tiende a resaltar regiones amplias de la imagen sin capturar con precisión los contornos finos de los objetos, lo que puede reducir su utilidad en escenarios donde se requiere un nivel de explicación más detallado.


#### Ejemplo 1 de Grad-CAM:clasificacion de imagenes con ResNet50 

**Figura 1**
Aplicación de Grad-CAM a ResNet50 en la tarea de clasificación de imágenes. En este caso, cada fila corresponde a una instancia diferente y cada columna a una clase presente en ImageNet junto con la probabilidad asignada por este modelo a esa instancia para dicha clase. Se puede observar cómo el mapa de calor generado por Grad-CAM resalta las regiones de la imagen que son más relevantes para la predicción de una clase específica, principalmente en la diagonal de la figura donde la imagen corresponde con la clase, pero a la vez fuera de la diagonal se puede ver como cambia la explicación cuando no hay correspondencia entre la clase y la imagen. Por ejemplo, cuando la imagen contiene un chita y se evalua la clase avión, el mapa de calor resalta una región en la parte superior de la imagen que contiene el cielo, lo que podría interpretarse como un feature que contribuye a la activación de la clase avión.

![Grad-CAM en ResNet50](./figures/gradcam_resnet50.png)

Grad-CAM demuestra que el modelo localiza con alta precisión los objetos correctos y mantiene el foco en el sujeto relevante incluso cuando discrimina entre clases semánticamente similares, como diferentes tipos de felinos, dirigiendo su atención por defecto al objeto más prominente cuando la clase es incorrecta. Más revelador aún, la herramienta expone las heurísticas internas y los sesgos del modelo, como cuando asocia la textura de la parrilla de un coche con un "gato atigrado" (tabby) o la luz brillante de una linterna de buzo con un "avión" (airliner). Así, Grad-CAM funciona como una potente herramienta de diagnóstico que no solo valida el razonamiento correcto del modelo, sino que también puede evidenciar sesgos en el comportamiento del modelo y las lógicas a veces inesperadas detrás de sus errores.

#### Ejemplo 2 de Grad-CAM con BLIP


**Figura 2** Como se puede aplicar Grad-CAM a modelos en tareas mas complejas como Captioning, aqui se ve la ultima capa de atencion del decoder de BLIP, y como se van generando los mapas de activacion para cada palabra del caption generado.

![Grad-CAM en Captioning](./figures/blip_gradcam_compact.png)

Podemos comprobar que Grad-CAM puede extenderse a tareas más complejas, como la generación de subtítulos, e incluso aplicarse a modelos multimodales basados en transformers. En este caso se utilizó el modelo BLIP, un sistema de visión y lenguaje que combina un codificador visual con un codificador textual para realizar tareas como image-text matching. La adaptación de Grad-CAM a BLIP permite localizar las regiones de la imagen relevantes para cada token del texto en dicha tarea.

La implementación se basa en:

Capa utilizada: un bloque de cross-attention del transformador multimodal (por defecto, el 6º).

Mecanismo:

Forward pass → cálculo del logit ITM.

Backward pass → obtención de gradientes sobre el cross-attention.

Gradientes × mapas de atención → generación de mapas Grad-CAM sobre los parches de la imagen.

Salida: mapas asociados a cada token y un mapa promedio a nivel de frase.

#### Ejemplo 3 de varias arquitecturas

**Figura 3** Comparación exhaustiva de la interpretabilidad de tres arquitecturas de redes neuronales de visión por computadora: ResNet50, VGG16 (representantes de las Redes Neuronales Convolucionales o CNNs) y ViT-B/16 (un Vision Transformer moderno). Se utilizan tres imágenes de instancia diversas para evaluar cómo cada modelo localiza la evidencia visual para cinco clases de destino diferentes de IMAGENET1k. Los mapas de calor (Grad-CAM) indican las regiones de la imagen que más influyeron en la decisión del modelo, mientras que el porcentaje (pred) debajo de cada mapa muestra la confianza (probabilidad Softmax) que el modelo asignó a esa clase específica para esa imagen.

![Grad-CAM en diferentes arquitecturas](./figures/gradcam_models.png)

Al comparar diferentes arquitecturas, Grad-CAM revela una dicotomía fundamental en los mecanismos de razonamiento visual: mientras las CNNs como ResNet50 y VGG16 destacan por su precisa localización espacial, basando su confianza en características intrínsecas del objeto (como se ve en el avión), el Vision Transformer (ViT) utiliza un enfoque de contexto global. Este último genera mapas de atención más difusos pero puede alcanzar una confianza sorprendentemente alta al interpretar la escena completa, como lo demuestra al clasificar el caballo basándose en el entorno de "animales en un campo" y al asociar fuertemente el avión con el cielo. En esencia, la visualización no muestra un fallo del ViT, sino su diferente estrategia de inferencia, más contextual y menos localizada que la de las CNNs.

Cabe destacar que la aplicación en este caso de Grad-CAM a Vision Transformers (ViT) es menos directa que en CNNs, ya que los ViT no tienen capas convolucionales tradicionales. Nosotros aplicamos el método a la salida del Decoder del ViT, que contiene información global de la imagen a través de los tokens de atención. Puede ser interesante explorar otras variantes de Grad-CAM diseñadas específicamente para Transformers, como **ViT-Grad-CAM** (ver: *https://github.com/Mikael17125/ViT-GradCAM*), que podrían ofrecer una mejor alineación con la arquitectura subyacente y mejorar la calidad de las explicaciones visuales.

# 4. Más allá de Grad-CAM

Con la intención de mejorar la definición y la estabilidad de los mapas de calor explicativos, se han propuesto diversas variantes de Grad-CAM. Estas buscan refinar la localización, reducir la dependencia de los gradientes o aumentar la resolución espacial de las visualizaciones. Entre ellas usaremos también una incluída en el trabajo original que es Guided Grad-CAM (2017). Esta técnica combina Grad-CAM con Guided Backpropagation para obtener mapas de calor de alta resolución que resaltan detalles finos y bordes, proporcionando una visualización más clara de las regiones relevantes para la predicción. A continuación, se describen algunas de las variantes más destacadas:

### Grad-CAM++ (2018)

* **Idea:** ajusta la forma de calcular los pesos $\alpha_k^c$ para tener en cuenta múltiples instancias de un mismo objeto en la imagen. En lugar de promediar los gradientes, utiliza una combinación ponderada que enfatiza las regiones con activaciones más fuertes utilizando derivadas de segundo orden.
* **Ventajas:** produce mapas más precisos en escenarios complejos y con objetos superpuestos.
* **Desventajas:** mayor costo computacional y complejidad en el cálculo.

### Score-CAM (2019)

* **Idea:** evita el uso directo de los gradientes, utilizando en su lugar las activaciones de la capa convolucional para generar mapas de calor. Cada mapa de activación se normaliza y se utiliza para ponderar la entrada, evaluando cómo cambia la predicción del modelo.
* **Ventajas:** no depende de los gradientes, lo que puede mejorar la estabilidad y la interpretabilidad.
* **Desventajas:** requiere múltiples evaluaciones del modelo, aumentando el costo computacional.

### HiRes-CAM (2020)

* **Idea:** combina Grad-CAM con técnicas de superresolución para mejorar la resolución espacial de los mapas de calor. Utiliza interpolación y refinamiento basado en los gradientes para obtener visualizaciones más detalladas.
* **Ventajas:** mapas de calor con mayor detalle y definición.
* **Desventajas:** mayor complejidad y costo computacional.

### Ablation-CAM (2020)

* **Idea:** se basa en la idea de ablation, eliminando sistemáticamente partes de la entrada para evaluar su impacto en la predicción. Genera mapas de calor al medir cómo cambia la salida del modelo cuando se ocultan diferentes regiones. Similar a la idea de Occlusion Maps, pero aplicado a las activaciones internas del modelo.
* **Ventajas:** proporciona una medida directa de la importancia de cada región.
* **Desventajas:** computacionalmente costoso y puede ser menos estable que los métodos basados en gradientes.

#### Comparación de métodos de explicabilidad basados en Grad-CAM

**Figura 4**: Comparación de cinco métodos de atribución visual (GradCAM, GradCAM++, HiResCAM, AblationCAM y Guided GradCAM) aplicados a una imagen de entrada compleja. Cada fila muestra los mapas de activación generados por un método diferente para cinco clases de destino de ImageNet ("Tiger Cat", "Golden Retriever", "Zebra", "Plane", "Trailer Truck"). La visualización permite evaluar y contrastar la precisión en la localización de objetos, la especificidad de clase y el nivel de detalle que ofrece cada técnica de interpretabilidad.

![Comparación de métodos de explicabilidad basados en Grad-CAM](./figures/gradcam_methods.png)

El análisis de los métodos basados en mapas de calor (GradCAM, GradCAM++, HiResCAM, AblationCAM) demuestra su gran eficacia para localizar correctamente los objetos de interés, como el gato y el perro, siendo las variantes más avanzadas como HiResCAM y AblationCAM las que ofrecen una mayor nitidez y fidelidad a los contornos. Más importante aún, estas técnicas revelan el proceso de razonamiento del modelo ante la incertidumbre: la atención se desvía a objetos semánticamente similares (los caballos por la "Zebra") o a características contextuales aprendidas cuando la clase está ausente, como resaltar el cielo para la clase "Plane". Esto expone los sesgos y las asociaciones que el modelo ha internalizado durante su entrenamiento.

En contraste, Guided GradCAM opera a un nivel de detalle mucho más profundo, respondiendo no a "dónde" mira el modelo, sino a "qué píxeles específicos" dentro de esa región son importantes. Al resaltar texturas finas como el pelaje del gato, los bigotes o los bordes, esta técnica ofrece una evidencia granular de las características exactas que fundamentan la decisión. En conjunto, la figura demuestra que mientras los mapas de calor son ideales para una evaluación de localización general, Guided GradCAM es una herramienta de "zoom" para un análisis forense de los atributos visuales que el modelo realmente utiliza.

# 5. Conclusión

A lo largo de este análisis, mostramos la importancia que tienen Grad-CAM y sus variantes más alla de una simple herramientas de visualización ya que se han convertido en instrumentos de diagnóstico indispensables para la auditoría y comprensión profunda de los modelos de visión por computadora. Las conclusiones extraídas de las figuras presentadas convergen en una idea central: la capacidad de Grad-CAM para "abrir la caja negra" nos permite entender no solo qué decide un modelo, sino cómo y por qué.

Si bien Grad-CAM es una aproximación y no el reflejo completo del complejo proceso de inferencia de una red, su capacidad para traducir las decisiones abstractas de un modelo en evidencia visual e interpretable es fundamental. Permite a los desarrolladores e investigadores no solo celebrar los éxitos, sino también entender, cuestionar y mitigar los fallos, identificar sesgos y, en última instancia, construir sistemas de inteligencia artificial más robustos, fiables y transparentes.

# **Referencias:**  

- Zeiler, M. D., & Fergus, R. (2014). *Visualizing and Understanding Convolutional Networks*. In ECCV. Springer.  
- Springenberg, J. T., Dosovitskiy, A., Brox, T., & Riedmiller, M. (2014). *Striving for Simplicity: The All Convolutional Net*. arXiv:1412.6806.
- Zhou, B., Khosla, A., Lapedriza, A., Oliva, A., & Torralba, A. (2016). *Learning Deep Features for Discriminative Localization*. CVPR. [Paper CAM](http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)
- Selvaraju, R. R., et al. (2017). *Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization*. ICCV. [Paper Grad-CAM](https://arxiv.org/abs/1610.02391)
- Chattopadhay, A., Sarkar, A., Howlader, P., & V. (2018). *Grad-CAM++: Improved Visual Explanations for Deep Convolutional Networks*. arXiv:1710.11063. [Paper Grad-CAM++](https://arxiv.org/abs/1710.11063)
- Montavon, G., Samek, W., & Müller, K. R. (2018). *Methods for interpreting and understanding deep neural networks*. Digital Signal Processing, 73, 1–15.  
- Samek, W., Montavon, G., Vedaldi, A., Hansen, L. K., & Müller, K. R. (Eds.). (2019). *Explainable AI: Interpreting, Explaining and Visualizing Deep Learning*. Springer.  
